In [0]:
import requests 
import pprint
import json

# Mount ADLS on ADB 
mountPoint = '/mnt/mount'
configs = {"fs.azure.account.auth.type": "OAuth",
          "fs.azure.account.oauth.provider.type": "org.apache.hadoop.fs.azurebfs.oauth2.ClientCredsTokenProvider",
          "fs.azure.account.oauth2.client.id": "xxxxxx-eb80-4b31-ad4c-xxxxx",
          "fs.azure.account.oauth2.client.secret": "xxxxx~1ZubNEbBgKbmvddLwh-xxxxx",
          "fs.azure.account.oauth2.client.endpoint": "https://login.microsoftonline.com/xx-xx-4c34-af63-xx/oauth2/token"}
if not any(mount.mountPoint == mountPoint for mount in dbutils.fs.mounts()):
    dbutils.fs.mount(
      source = "abfss://news@jijosource.dfs.core.windows.net/",
      mount_point = "/mnt/mount",
      extra_configs = configs)

# map variable url to fetch 'Everything' from newsAPI
url = 'https://newsapi.org/v2/everything?'

# filter out news about 'data engineering' specifically and restrict the page size
# generate apiKey from https://newsapi.org/
parameters = {
    'q': 'data engineering',
    'pageSize': 20,
    'apiKey': 'xxxxxx'
}

# create a JSON datset to store the filtered news from newsAPI.
news_json = requests.get(url, params=parameters).json()

#Print the json just to visualize
pprint.pprint(news_json)

# Dump the data in ADLS Gen 2
j = json.dumps(news_json)
with open("/dbfs/mnt/mount/news.json", "w") as f:
    f.write(j)
    f.close()